# Setup Environment and Load Libraries
Import necessary libraries such as TensorFlow, Keras, NumPy, scikit-learn, and any other utilities.

In [10]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV3Small, EfficientNetB7
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# 1. Setup Data Paths and Parameters
data_dir = './app/fer2013_data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
img_height, img_width = 224, 224  # Standard image size for these models
batch_size = 32
epochs = 10
num_classes = 1 # Binary classification: Happy or Not Happy
weights_dir = './weights'
os.makedirs(weights_dir, exist_ok=True)

# Define Data Paths and Parameters
Define the paths to the training and testing data directories, image size, batch size, and model names.

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

def binary_data_generator(data_dir, img_height, img_width, batch_size, datagen):
    """
    Creates a data generator for binary classification (Happy vs. Not Happy).
    """
    happy_dir = os.path.join(data_dir, 'Happy')
    other_emotions = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d)) and d != 'Happy']

    image_list = []
    labels = []

    # Load happy images
    for filename in os.listdir(happy_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_list.append(os.path.join(happy_dir, filename))
            labels.append(1)  # 1 for Happy

    # Load other emotion images
    for emotion in other_emotions:
        emotion_dir = os.path.join(data_dir, emotion)
        for filename in os.listdir(emotion_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_list.append(os.path.join(emotion_dir, filename))
                labels.append(0)  # 0 for Not Happy

    # Convert to numpy arrays
    image_list = np.array(image_list)
    labels = np.array(labels)

    # Create a DataFrame
    df = pd.DataFrame({'image': image_list, 'label': labels})

    # Create the generator
    generator = datagen.flow_from_dataframe(
        dataframe=df,
        x_col='image',
        y_col='label',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='raw',  # Important for binary classification
        shuffle=True
    )

    return generator

# Create data generators
train_data = binary_data_generator(train_dir, img_height, img_width, batch_size, train_datagen)
test_data = binary_data_generator(test_dir, img_height, img_width, batch_size, test_datagen)


Found 35924 validated image filenames.
Found 8952 validated image filenames.


# Data Preprocessing
Create a function to load and preprocess images. This includes resizing, converting to grayscale (if needed), and normalizing pixel values.

In [12]:
def create_model(base_model, num_classes):
    """
    Creates a model with a custom classification head.
    """
    base_model.trainable = False  # Freeze the base model layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)  # Sigmoid for binary classification
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Dictionary of models
models = {
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
    'MobileNetV3': MobileNetV3Small(include_top=False, input_shape=(img_height, img_width, 3)),
    'EfficientNetB7': EfficientNetB7(include_top=False, input_shape=(img_height, img_width, 3))
}

# Create and compile models
compiled_models = {}
for model_name, base_model in models.items():
    model = create_model(base_model, num_classes)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy']) # Binary crossentropy
    compiled_models[model_name] = model


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Load and Preprocess Images
Use the image preprocessing function to load the training and testing datasets. Create data generators for efficient batch processing.

In [16]:
results = []

for model_name, model in compiled_models.items():
    print(f"Evaluating model: {model_name}")
    
    weights_path = os.path.join(weights_dir, f'{model_name}.h5')

    if os.path.exists(weights_path):
        print(f"Loading weights from {weights_path}")
        model.load_weights(weights_path)
    else:
        print(f"Training {model_name}...")
        start_time = time.time()
        history = model.fit(
            train_data,
            epochs=epochs,
            validation_data=test_data
        )
        end_time = time.time()
        training_time = end_time - start_time
        print(f"Training time for {model_name}: {training_time:.2f} seconds")

        # Save weights after training
        model.save_weights(weights_path)
        print(f"Weights saved to {weights_path}")

    # Measure inference time
    start_time = time.time()
    predictions = model.predict(test_data, verbose=1)
    end_time = time.time()
    inference_time = (end_time - start_time) / len(test_data.filenames)

    # Convert predictions to binary labels
    predicted_labels = (predictions > 0.5).astype(int).flatten()
    true_labels = test_data.classes

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    auc = roc_auc_score(true_labels, predictions)

    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'F1-Score': f1,
        'AUC': auc,
        'Inference Time (s/image)': inference_time
    })

    print(f"Classification Report for {model_name}:\n")
    print(classification_report(true_labels, predicted_labels))


Evaluating model: VGG16
Training VGG16...
Epoch 1/10
 213/1123 ━━━━━━━━━━━━━━━━━━━━ 46:54 3s/step - accuracy: 0.7794 - loss: 0.5518

KeyboardInterrupt: 

# Define and Load Models
Define the model architectures (VGG16, ResNet50, MobileNetV3, EfficientNetB7) and load pre-trained weights (if available). Modify the output layer for binary classification (Happy vs. Not Happy).

In [7]:
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV3Large, EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Function to define and load models with modified output layers
def define_models(image_size):
    """
    Define model architectures and modify output layers for binary classification.
    
    Args:
        image_size (tuple): Input image size (height, width).
    
    Returns:
        dict: Dictionary of models with their names as keys.
    """
    models = {}

    # VGG16
    base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    x = GlobalAveragePooling2D()(base_model_vgg16.output)
    x = Dense(1, activation='sigmoid')(x)  # Binary classification
    models['VGG16'] = Model(inputs=base_model_vgg16.input, outputs=x)

    # ResNet50
    base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    x = GlobalAveragePooling2D()(base_model_resnet50.output)
    x = Dense(1, activation='sigmoid')(x)  # Binary classification
    models['ResNet50'] = Model(inputs=base_model_resnet50.input, outputs=x)

    # MobileNetV3
    base_model_mobilenetv3 = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    x = GlobalAveragePooling2D()(base_model_mobilenetv3.output)
    x = Dense(1, activation='sigmoid')(x)  # Binary classification
    models['MobileNetV3'] = Model(inputs=base_model_mobilenetv3.input, outputs=x)

    # EfficientNetB7
    base_model_efficientnetb7 = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    x = GlobalAveragePooling2D()(base_model_efficientnetb7.output)
    x = Dense(1, activation='sigmoid')(x)  # Binary classification
    models['EfficientNetB7'] = Model(inputs=base_model_efficientnetb7.input, outputs=x)

    return models

# Define models
models = define_models(image_size)

# Print model summaries (optional, for debugging purposes)
for model_name, model in models.items():
    print(f"Model: {model_name}")
    model.summary()

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step
Model: VGG16


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,201 (56.13 MB)

 Trainable params: 14,715,201 (56.13 MB)

...

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling_1[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_2[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │      1,728 │ stem_conv_pad[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      2,048 │ block1a_se_excit

 Total params: 64,100,248 (244.52 MB)

 Trainable params: 63,789,521 (243.34 MB)

 Non-trainable params: 310,727 (1.19 MB)

# Model Evaluation
Iterate through each model, load its weights, and evaluate it on the test dataset. Calculate accuracy, precision, F1-score, AUC, and inference time per image.

In [15]:
# Evaluate models on the test dataset
results = []

for model_name, model in models.items():
    print(f"Evaluating model: {model_name}")
    
    # Load pre-trained weights (assuming weights are saved in './weights/{model_name}.h5')
    weights_path = f'./weights/{model_name}.h5'
    if os.path.exists(weights_path):
        model.load_weights(weights_path)
    else:
        print(f"Warning: Weights for {model_name} not found at {weights_path}. Skipping evaluation.")
        continue

    # Measure inference time per image
    start_time = time.time()
    predictions = model.predict(test_data, verbose=1)
    end_time = time.time()
    inference_time = (end_time - start_time) / len(test_data.filenames)

    # Convert predictions to binary labels
    predicted_labels = (predictions > 0.5).astype(int).flatten()
    true_labels = test_data.classes

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    auc = roc_auc_score(true_labels, predictions)

    # Append results
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'F1-Score': f1,
        'AUC': auc,
        'Inference Time (s/image)': inference_time
    })

    # Print classification report (optional)
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(true_labels, predicted_labels))

# Display results
import pandas as pd
results_df = pd.DataFrame(results)
print("\nModel Evaluation Results:")
print(results_df)

Evaluating model: VGG16
Evaluating model: ResNet50
Evaluating model: MobileNetV3
Evaluating model: EfficientNetB7

Model Evaluation Results:
Empty DataFrame
Columns: []
Index: []


# Aggregate Results
Compile the evaluation metrics for each model into a table or dictionary for comparison.

In [14]:
results_df = pd.DataFrame(results)
print("\nModel Evaluation Results:")
print(results_df)


Model Evaluation Results:
Empty DataFrame
Columns: []
Index: []
